In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from scipy.ndimage import morphology
from skimage.measure import label, regionprops
from skimage.filters import threshold_triangle

In [9]:
def toGray(image):
    return (0.2989*image[:, :, 0]+0.587*image[:, :, 1]+0.114 * image[:, :, 2]).astype('uint8')

 
def binarisation(image, limit_min, limit_max):
    B = image.copy()
    B[B <= limit_min] = 0
    B[B >= limit_max] = 0
    B[B > 0] = 1
    return B

def count(image):
    gray = toGray(image)
    thresh = threshold_triangle(gray)
    binary = binarisation(gray, 0, thresh)

    binary = morphology.binary_dilation(binary, iterations=5)

    labeled = label(binary)
    print("Max: ", np.max(labeled))

    areas = []
    for region in regionprops(labeled):
        areas.append(region.area)

    for region in regionprops(labeled):
        if region.area < np.mean(areas):
            labeled[labeled == region.label] = 0
        bbox = region.bbox
        if bbox[0] == 0 or bbox[1] == 0:
            labeled[labeled == region.label] = 0

    labeled[labeled > 0] = 1
    labeled = label(labeled)

    i = 1
    n = 0
    for region in regionprops(labeled):
        if circularity(region, i) > 100 and  region.area > 330000 and region.area < 700000:
            n+=1
        i+=1
    
    print("pencils:", n)
    return n

def circularity(region,label=1):
    return (region.perimeter ** 2) / region.area

all_pens = 0

for p in range(1, 13):
    constr = "img (" + str(p) + ").jpg"
    print(constr)
    image = plt.imread(constr)
    all_pens = all_pens + count(image)

print("all pencils in every picture", all_pens)

img (1).jpg
Max:  141
pencils: 0
img (2).jpg
Max:  135
pencils: 1
img (3).jpg
Max:  112
pencils: 1
img (4).jpg
Max:  148
pencils: 2
img (5).jpg
Max:  136
pencils: 2
img (6).jpg
Max:  216
pencils: 3
img (7).jpg
Max:  257
pencils: 3
img (8).jpg
Max:  370
pencils: 1
img (9).jpg
Max:  99
pencils: 2
img (10).jpg
Max:  194
pencils: 2
img (11).jpg
Max:  111
pencils: 3
img (12).jpg
Max:  135
pencils: 1
all pencils in every picture 21
